**NanoGPT : Code from scratch**

This is part of understanding "transformer" architecture (Decoder in this case) by implementing it from the scratch without using prefedined decoder architecture.

**Credit**: Andrej Karpathy - Let's build GPT: from scratch, in code, spelled out (https://www.youtube.com/watch?v=kCc8FmEb1nY)

In [1]:
# imports
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
# Downlaoding tiny shakespeare dataset as used in the video
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-12-24 17:53:09--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.02s   

2023-12-24 17:53:10 (56.7 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [26]:
# hyper-parameters
batch_size = 16
block_size = 32
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 128
n_head = 64
n_layer = 4
dropout = 0.0

torch.manual_seed(1337)

# Loading the dataset
with open('input.txt', 'r', encoding = 'utf-8') as f:
  text = f.read()

# Build a tokenizer
## Collect all the unique characters ocurring in the dataset.
chars = sorted(list(set(text)))
vocab_size = len(chars)

## Create a mapping from characters to integers and vice-versa
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

## Create lambda functions
## function_name = lambda variable: operation
### take input as string and return a list of integers
encode = lambda s: [stoi[c] for c in s]
### take list of integers as input and return a string
decode = lambda l: ' '.join([itos[i] for i in l])

# Creating training and test splits
data = torch.tensor((encode(text)), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

# Load data
def get_batch(split):
  data = train_data if split == "train" else val_data
  # generate random integers from the torch.randint(low=0, high=(data-blocksize)
  # , output tensor shape = (batchsize,1) )
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x, y = x.to(device), y.to(device)
  return x,y

# Implementing single head attention
class Head(nn.Module):
  """ single head attention """

  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(in_features=n_embd, out_features=head_size, bias=False)
    self.query = nn.Linear(in_features=n_embd, out_features=head_size, bias=False)
    self.value = nn.Linear(in_features=n_embd, out_features=head_size, bias=False)
    '''
    register_buffer(name, tensor, persistent=True)[SOURCE]
    Adds a buffer to the module.
    This is typically used to register a buffer that should not to be considered a model parameter.
    For example, BatchNorm’s running_mean is not a parameter, but is part of the module’s state.
    '''
    self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B,T,C = x.shape
    k = self.key(x)
    q = self.query(x)
    v = self.value(x)
    # Compute the affinities
    wei = q @ k.transpose(-2,-1) * (C**-0.5)
    ''' tensor.masked_fill(mask, value) '''
    wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf'))
    wei = F.softmax(wei, dim=-1)
    wei = self.dropout(wei)
    out = wei @ v
    return out

# Extending it to Multi-Head Attention
class MultiHeadAttention(nn.Module):
  """ multiple heads of self-attention in parallel """

  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.projection = nn.Linear(n_embd, n_embd)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)
    out = self.dropout(self.projection(out))
    return out

# Implementing a feed forward network to apply after attention.
# (inverse bottleneck)
class FeedForward(nn.Module):
  """ a linear layer followed by a non-linearity """

  def __init__(self, n_embd):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embd, 4*n_embd),
        nn.ReLU(),
        nn.Linear(4*n_embd, n_embd),
        nn.Dropout(dropout),
    )

  def forward(self, x):
    return self.net(x)

# Implementing a decoder block
class Block(nn.Module):
  """ Transformer decoder block: attention followed by linear layer """
  def __init__(self, n_embd, n_head):
    super().__init__()
    head_size = n_embd // n_head
    self.sa = MultiHeadAttention(n_head, head_size)
    self.ffwd = FeedForward(n_embd)
    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)

  def forward(self, x):
    x = x + self.sa(self.ln1(x))
    x = x + self.ffwd(self.ln2(x))
    return x

# Implementing transformer decoder model
class Decoder(nn.Module):
  """ Transformer decoder architecture """

  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
    self.ln_f = nn.LayerNorm(n_embd)
    self.lm_head = nn.Linear(n_embd, vocab_size)

  def forward(self, idx, targets=None):
    B, T = idx.shape
    tok_emb = self.token_embedding_table(idx) # B,T,C
    pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # T,C
    # Same position values for each batch
    x = tok_emb + pos_emb
    x = self.blocks(x)
    x = self.ln_f(x)
    logits = self.lm_head(x)

    if targets is None:
      loss = None
    else:
      B,T,C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    # idx (B,T)
    for _ in range(max_new_tokens):
      # crop idx to (block_size) length as position_ids till that is available
      idx_cond = idx[:, -block_size:]
      # get predictions
      logits, loss = self(idx_cond)
      # focus on last time-step OR last token
      logits = logits[:, -1, :] # B,T,C becomes B,C
      # apply softmax for probabilities
      probs = F.softmax(logits, dim=-1) #B,C
      # Instead of picking top, sample from distribution
      idx_next = torch.multinomial(probs, num_samples=1) #B,1
      # append sampled index to the running sequence
      idx = torch.cat((idx, idx_next), dim=1) #B,T+1
    return idx

# Running the decoder block
model = Decoder()
model = model.to(device)

# Num of params
print(sum(p.numel() for p in model.parameters()))

# Create a optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Calculate training and validation loss
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X,Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

# Training step
for iter in range(max_iters):
  # every interval eval training and validation loss
  if iter % eval_interval == 0 or iter == max_iters-1:
    losses = estimate_loss()
    print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  # sample a batch of data
  xb, yb = get_batch('train')

  # evaluate loss
  logits, loss = model(xb,yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

# generate from the model
bos = torch.zeros((1,1), dtype = torch.long, device=device)
print(decode(model.generate(bos, max_new_tokens=2000)[0].tolist()))

812609
step 0: train loss 4.3210, val loss 4.3210
step 100: train loss 2.5376, val loss 2.5389
step 200: train loss 2.4394, val loss 2.4524
step 300: train loss 2.3774, val loss 2.3876
step 400: train loss 2.3189, val loss 2.3355
step 500: train loss 2.2658, val loss 2.2863
step 600: train loss 2.2172, val loss 2.2390
step 700: train loss 2.1687, val loss 2.1933
step 800: train loss 2.1125, val loss 2.1471
step 900: train loss 2.0755, val loss 2.1304
step 1000: train loss 2.0397, val loss 2.1158
step 1100: train loss 2.0025, val loss 2.0787
step 1200: train loss 1.9650, val loss 2.0604
step 1300: train loss 1.9308, val loss 2.0375
step 1400: train loss 1.9184, val loss 2.0239
step 1500: train loss 1.8933, val loss 2.0084
step 1600: train loss 1.8676, val loss 1.9836
step 1700: train loss 1.8428, val loss 1.9716
step 1800: train loss 1.8264, val loss 1.9780
step 1900: train loss 1.8059, val loss 1.9282
step 2000: train loss 1.7951, val loss 1.9454
step 2100: train loss 1.7931, val loss 